<div style="color:#fff; background:#0070c0; font-weight:bold; border:2px solid #0070c0; padding:10px; border-radius:6px;">
**Purpose:** This notebook demonstrates how to use <span style="color:yellow;">Spark Expectations</span> with <span style="color:yellow;"> Slack by sending notifications`</span>.<br>
Its main focus is to show how data quality alerts and results can be sent to Slack channels.</span>
</div>

### Spark - Expectations - User - Guide - Documentation

<div style="color:red; font-weight:bold; border:2px solid red; padding:8px;">
⚠️ ALERT: Notebook meant to be ran by spinning up local docker compose (containers/compose.yaml) !
</div>

* Please read through the [Spark Expectation Documentation](https://engineering.nike.com/spark-expectations) before proceeding with this demo

#### widgets 
* `catalog`, `schema` - leave default values 
  * Tables are going to be prefixed with value provided in user widget text field

<div style="color:orange; font-weight:regular; border:1px solid orange; padding:8px;">
⚠️ Container comes with SparkExpectation by default. If SE version is overriden Kernel will need to be restarted!
</div>

* `Override SE version` check box to install different SparkExpectation library version
* `library_source` combo box defines library url(git branch or pypi) from where to pull library 
  * `pypi` ( installs latest published version available in PyPi)
  * `git` ( installs library from specified git branch)
    * Set `git_branch` input field to match git branch (example `main`)  

# Method 1: Use Webhook URL for Notebook Testing

This is used by default for the example notebook. This will just use the webhook URL throughout the notebook for a quick and easy usage.

In [ ]:
# For local Spark environment, we'll use webhook URL directly
# This section would be used in Databricks to create secret scopes

# Note: In a real Databricks environment, you would use:
# from databricks.sdk import WorkspaceClient
# from pyspark.errors import PySparkException
# w = WorkspaceClient()
# w.secrets.list_scopes()

# For local testing, we'll just use the webhook URL directly in the configuration
print("Running in local Spark environment - using webhook URL directly")
webhook_url = "https://hooks.slack.com/services/YOUR_WORKSPACE/YOUR_CHANNEL/YOUR_SECRET_TOKEN"  # Replace with your actual webhook URL
print(f"Webhook URL configured: {webhook_url[:50]}...")

Running in local Spark environment - using webhook URL directly
Webhook URL configured: https://hooks.slack.com/services/YOUR/WEBHOOK/URL

# Widget Setup

Widgets used in this notebook will be created and then set.

In [13]:
import ipywidgets as widgets
from IPython.display import display

import re
import pandas as pd

In [14]:
# GENERATE INPUT WIDGETS 

import ipywidgets as widgets
from IPython.display import display


widget_user = widgets.Text(
    value='user',
    placeholder='Type something',
    description='user: ',
    disabled=False,
    style={'description_width': '100px'}    
)

widget_git_org = widgets.Text(
    value='catalog_name',
    placeholder='Type something',
    description='git_org ',
    disabled=False,
    style={'description_width': '100px'}    
)

widget_catalog = widgets.Text(
    value='development',
    placeholder='Type something',
    description='catalog:',
    disabled=False,
    style={'description_width': '100px'}    
)

widget_schema = widgets.Text(
    value='team_name',
    placeholder='Type something',
    description='schema:',
    disabled=False,
    style={'description_width': '100px'}
)

widget_library_source = widgets.Combobox(
    placeholder='Choose source',
    options=['pypi', 'git'],
    description='library_source:',
    ensure_option=True,
    value='git',
    disabled=False,
    style={'description_width': '100px'}
)

widget_git_branch_or_commit = widgets.Text(
    value='main',
    placeholder='Type branch name or commit hash',
    description='git_branch_or_commit:',
    disabled=False,
    style={'description_width': '150px'}
)

widget_override_version = widgets.Checkbox(
    value=False,
    description='Override SE version',
    disabled=False,
    style={'description_width': '30px'}
    
)

hbox = widgets.HBox([
    widget_user,
    widget_catalog, 
    widget_schema,
    widget_override_version, 
    widget_library_source, 
    widget_git_org,
    widget_git_branch_or_commit
])

# Display widgets
display(hbox)

In [15]:
import re
import pandas as pd

user = re.sub(r'[^a-zA-Z]', '', widget_user.value).lower()
catalog = widget_catalog.value
schema = widget_schema.value
override_se_version = widget_override_version.value
library = widget_library_source.value
org = widget_git_org.value
branch_or_commit = widget_git_branch_or_commit.value

CONFIG = {
    "owner": user,
    "catalog": "default",  # Use default catalog for local Spark
    "schema": "default",   # Use default schema for local Spark
    "user": user,
    "product_id": f"se_{user}_product",
    "in_memory_source": f"se_{user}_source",
    "rules_table": f"se_{user}_rules",     # Simple table names for local Spark
    "stats_table": f"se_{user}_stats",    # Simple table names for local Spark
    "target_table": f"se_{user}_target",  # Simple table names for local Spark
    "override_se_version" : override_se_version,
    "library": library,
    "org": org,
    "branch_or_commit": branch_or_commit
}

config_df = pd.DataFrame(list(CONFIG.items()), columns=['Key', 'Value'])

# Install Spark Expectation

If Running from local container it will come with latest spark-expectation library

In [16]:
# Override Spark Expectations based on user input
if override_se_version:
    print("-----OVERRIDING SPARK-EXPECTATIONS VERSION")
    if CONFIG["library"] == "pypi":
      print("-----INSTALLING SPARK-EXPECTATIONS from PyPi")
      %pip install spark-expectations
    elif CONFIG["library"] == "git":
      print(f"-----INSTALLING SPARK-EXPECTATIONS from Git Org/User {CONFIG['org']}, Branch/Commit {CONFIG['branch_or_commit']}")
      giturl = f"git+https://github.com/{CONFIG['org']}/spark-expectations.git@{CONFIG['branch_or_commit']}"
      %pip install --force-reinstall {giturl}    
else:
    print(f"---- Using SparkExpectation from local codebase")

---- Using SparkExpectation from local codebase


In [17]:
# CREATE SPARK SESSION AND DATABASE
from pyspark.sql import SparkSession

# Create or get a Spark session
spark = SparkSession.builder \
    .appName("Spark SQL Example") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0") \
    .getOrCreate()

In [18]:
# Use default database for local Spark environment
spark.sql("USE default")

# Verify the schema was created
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [19]:
databases_df = spark.sql("SHOW DATABASES")
databases_df.show(truncate=False)

tables_df = spark.sql("SHOW TABLES")
tables_df.show(truncate=False)

+---------+
|namespace|
+---------+
|default  |
+---------+

+---------+-------------+-----------+
|namespace|tableName    |isTemporary|
+---------+-------------+-----------+
|default  |se_user_rules|false      |
+---------+-------------+-----------+



In [20]:
# Clean up tables from previous runs
pattern = f"se_{CONFIG['user']}*"

# Set the current database to default
spark.sql("USE default")

# Drop tables matching pattern - for local Spark, we don't need to specify database
tables_df = spark.sql(f"SHOW TABLES LIKE '{pattern}'")
tables_to_drop = [row for row in tables_df.collect() if not row["isTemporary"]]

if tables_to_drop:
    print(f"Found {len(tables_to_drop)} tables to drop.")
    for row in tables_to_drop:
        table_name = row["tableName"]
        spark.sql(f"DROP TABLE IF EXISTS {table_name}")
        print(f"Dropped table: {table_name}")
else:
    print("----- No tables to drop")

Found 1 tables to drop.
Dropped table: se_user_rules


In [21]:
# Clean up views from previous runs
views_df = spark.sql(f"SHOW VIEWS LIKE '{pattern}'")
views_to_drop = views_df.collect()

if views_to_drop:
    print(f"Found {len(views_to_drop)} views to drop.")
    for row in views_to_drop:
        view_name = row["viewName"]
        spark.sql(f"DROP VIEW IF EXISTS {view_name}")
        print(f"Dropped view: {view_name}")
else:
    print("----- No views to drop")

----- No views to drop


In [22]:
# Getting Started with Spark Expectations: Simple Example

## 1. Sample Source Dataset
# initialize simple Pandas DataFrame and convert it to a Spark DataFrame

import pandas as pd
from pyspark.sql import SparkSession


## 2. Define Simple `row_dq` Rules
# Create a rules DataFrame with a few simple data quality rules

rules_data = [
    {
        "product_id": CONFIG["product_id"],
        "table_name": CONFIG["target_table"],
        "rule_type": "row_dq",
        "rule": "age_not_null",
        "column_name": "age",
        "expectation": "age IS NOT NULL",
        "action_if_failed": "warn",
        "tag": "completeness",
        "description": "Age must not be null",
        "enable_for_source_dq_validation": True,
        "enable_for_target_dq_validation": True,
        "is_active": True,
        "enable_error_drop_alert": False,
        "error_drop_threshold": 0,
        "priority": "medium",
    },
    {
        "product_id": CONFIG["product_id"],
        "table_name": CONFIG["target_table"],
        "rule_type": "row_dq",
        "rule": "age_adult",
        "column_name": "age",
        "expectation": "age < 20",
        "action_if_failed": "ignore",
        "tag": "validity",
        "description": "Age must be less than 20",
        "enable_for_source_dq_validation": True,
        "enable_for_target_dq_validation": True,
        "is_active": True,
        "enable_error_drop_alert": False,
        "error_drop_threshold": 0,
        "priority": "medium",
    },
        {
        "product_id": CONFIG["product_id"],
        "table_name": CONFIG["target_table"],
        "rule_type": "row_dq",
        "rule": "email_not_null",
        "column_name": "email",
        "expectation": "email IS NOT NULL",
        "action_if_failed": "drop",
        "tag": "completeness",
        "description": "Email must not be null",
        "enable_for_source_dq_validation": True,
        "enable_for_target_dq_validation": True,
        "is_active": True,
        "enable_error_drop_alert": False,
        "error_drop_threshold": 0,
        "priority": "medium",
    }

    
]
rules_df = spark.createDataFrame(pd.DataFrame(rules_data))
rules_df.write.format("delta").mode("overwrite").saveAsTable(CONFIG['rules_table'])

rules_df.show(truncate=False)

+---------------+--------------+---------+--------------+-----------+-----------------+----------------+------------+------------------------+-------------------------------+-------------------------------+---------+-----------------------+--------------------+--------+
|product_id     |table_name    |rule_type|rule          |column_name|expectation      |action_if_failed|tag         |description             |enable_for_source_dq_validation|enable_for_target_dq_validation|is_active|enable_error_drop_alert|error_drop_threshold|priority|
+---------------+--------------+---------+--------------+-----------+-----------------+----------------+------------+------------------------+-------------------------------+-------------------------------+---------+-----------------------+--------------------+--------+
|se_user_product|se_user_target|row_dq   |age_not_null  |age        |age IS NOT NULL  |warn            |completeness|Age must not be null    |true                           |true         

# Notification Configurations for Slack

This is where you can set the webhook URL with the following ways:
- setting the webhook URL manually
- using databricks secret storage (assuming you have secrets stored within your notebook)
- using cerberus secrets storage 

In [ ]:
# Configure streaming and notification configuration
from spark_expectations.config.user_config import Constants as user_config

# This is a dictionary that can be used to configure Spark Expectations behavior and override default settings.
stats_streaming_config_dict = {
    user_config.se_enable_streaming: False,
}

# For local Spark environment, we don't need Databricks workspace info
# dbx_workspace_id = get_context().workspaceId
# dbx_workspace_url = get_context().browserHostName

user_conf_dict = {
    user_config.se_notifications_enable_slack: True,
    # Slack Configuration - where you supply the webhook URL for your Slack channel
    user_config.se_notifications_slack_webhook_url: "https://hooks.slack.com/services/YOUR_WORKSPACE/YOUR_CHANNEL/YOUR_SECRET_TOKEN",  # Replace with your actual webhook URL

    # Fill in your cbs_url + cbs_sdb_path on where your secret is stored.
    # user_config.secret_type: "cerberus",
    # user_config.cbs_url: "https://cerberus.com",
    # user_config.cbs_sdb_path: "app/your/sdb/path",

    # Optionally configure additional Slack settings
    # user_config.se_notifications_slack_channel: "#data-quality-alerts",
    # user_config.se_notifications_slack_username: "Spark Expectations Bot",
    # user_config.se_notifications_slack_icon_emoji: ":warning:",

    # Enable detailed results in notifications
    user_config.se_enable_query_dq_detailed_result: True,
    
    # Set notification on query failure - corrected attribute name
    user_config.se_notifications_on_error_drop_exceeds_threshold_breach: True,
    user_config.se_notifications_on_error_drop_threshold: 15
}

In [42]:
# Additional notification configuration options for local environment
# Fill in your workspace url + secret scope for Databricks (not needed for local).
# user_config.secret_type: "databricks",
# user_config.dbx_workspace_url: dbx_workspace_url,
# user_config.dbx_secret_scope: "slack",

# Add notification triggers to user_conf_dict
user_conf_dict.update({
    # Notification triggers - ENABLE THESE TO GET SLACK NOTIFICATIONS
    user_config.se_notifications_on_start: True,                              # ✅ Notify when job starts
    user_config.se_notifications_on_completion: True,                         # ✅ Notify when job completes
    user_config.se_notifications_on_fail: True,                              # ✅ Notify on failures
    user_config.se_notifications_on_error_drop_exceeds_threshold_breach: True, # ✅ Notify when error threshold breached
    user_config.se_notifications_on_rules_action_if_failed_set_ignore: True,  # ✅ Notify when 'ignore' rules fail
    user_config.se_notifications_on_error_drop_threshold: 1,                  # Low threshold (1%) to trigger notifications
})

print("✅ Updated notification configuration with all triggers ENABLED:")
print("- on_start: True")
print("- on_completion: True") 
print("- on_fail: True")
print("- on_error_drop_exceeds_threshold_breach: True")
print("- on_rules_action_if_failed_set_ignore: True")
print("- error_drop_threshold: 1%")
print("\nFull configuration:")
display(user_conf_dict)

✅ Updated notification configuration with all triggers ENABLED:
- on_start: True
- on_completion: True
- on_fail: True
- on_error_drop_exceeds_threshold_breach: True
- on_rules_action_if_failed_set_ignore: True
- error_drop_threshold: 1%

Full configuration:


{'spark.expectations.notifications.slack.enabled': True,
 'spark.expectations.notifications.slack.webhook.url': 'https://hooks.slack.com/services/YOUR/WEBHOOK/URL',
 'spark.expectations.query.dq.detailed.stats': True,
 'spark.expectations.notifications.on.error.drop.exceeds.threshold.breach': True,
 'spark.expectations.notifications.error.drop.threshold': 1,
 'spark.expectations.notifications.on.start': True,
 'spark.expectations.notifications.on.completion': True,
 'spark.expectations.notifications.on.fail': True,
 'spark.expectations.notifications.on.rules.action.if.failed.set.ignore': True}

In [43]:
## 3. Run Spark Expectations

from pyspark.sql import DataFrame

from spark_expectations.core import load_configurations

from spark_expectations.core.expectations import (
    SparkExpectations,
    WrappedDataFrameWriter,
)


writer = WrappedDataFrameWriter().mode("overwrite").format("delta")


# Initialize Default Config 
load_configurations(spark) 

"""
This class implements/supports running the data quality rules on a dataframe returned by a function

Args:
    product_id: Name of the product
    rules_df: DataFrame which contains the rules. User is responsible for reading
        the rules_table in which ever system it is
    stats_table: Name of the table where the stats/audit-info need to be written
    debugger: Mark it as "True" if the debugger mode need to be enabled, by default is False
    stats_streaming_options: Provide options to override the defaults, while writing into the stats streaming table
"""
se = SparkExpectations(
    product_id=CONFIG["product_id"],
    rules_df=rules_df,
    stats_table=CONFIG["stats_table"],
    stats_table_writer=writer,
    target_and_error_table_writer=writer,
    stats_streaming_options=stats_streaming_config_dict,
)

#  Initialize input data
data = [
    {"id": 1, "age": 19,   "email": "alice@example.com"},
    {"id": 2, "age": 17,   "email": "bob@example.com"},
    {"id": 3, "age": None, "email": "charlie@example.com"},
    {"id": 4, "age": 40,   "email": "mike@example.com"},
    {"id": 5, "age": None, "email": "ron@example.com"},
    {"id": 6, "age": 35,   "email": None},
]
input_df = spark.createDataFrame(pd.DataFrame(data))
input_df.show(truncate=False)

"""
This decorator helps to wrap a function which returns dataframe and apply dataframe rules on it

Args:
    target_table: Name of the table where the final dataframe need to be written
    write_to_table: Mark it as "True" if the dataframe need to be written as table
    write_to_temp_table: Mark it as "True" if the input dataframe need to be written to the temp table to break
                        the spark plan
    user_conf: Provide options to override the defaults, while writing into the stats streaming table
    target_table_view: This view is created after the _row_dq process to run the target agg_dq and query_dq.
        If value is not provided, defaulted to {target_table}_view
    target_and_error_table_writer: Provide the writer to write the target and error table,
        this will take precedence over the class level writer

Returns:
    Any: Returns a function which applied the expectations on dataset
"""


@se.with_expectations(
    target_table=CONFIG["target_table"],
    write_to_table=True,
    write_to_temp_table=True,
    user_conf=user_conf_dict,
)
def get_dataset():
    _df_source: DataFrame = input_df
    _df_source.createOrReplaceTempView(CONFIG["in_memory_source"])
    return _df_source


# This will run the DQ checks and raise if any "fail" rules are violated
get_dataset()

25/10/17 15:05:06 WARN CacheManager: Asked to cache already cached data.


+---+----+-------------------+
|id |age |email              |
+---+----+-------------------+
|1  |19.0|alice@example.com  |
|2  |17.0|bob@example.com    |
|3  |NaN |charlie@example.com|
|4  |40.0|mike@example.com   |
|5  |NaN |ron@example.com    |
|6  |35.0|NULL               |
+---+----+-------------------+

[2025-10-17 15:05:09,660] [INFO] [spark_expectations] {slack.py:send_notification:42} - Message posted successfully!
[2025-10-17 15:05:09,664] [INFO] [spark_expectations] {expectations.py:wrapper:320} - The function dataframe is getting created
[2025-10-17 15:05:09,660] [INFO] [spark_expectations] {slack.py:send_notification:42} - Message posted successfully!
[2025-10-17 15:05:09,664] [INFO] [spark_expectations] {expectations.py:wrapper:320} - The function dataframe is getting created
[2025-10-17 15:05:09,933] [INFO] [spark_expectations] {expectations.py:wrapper:336} - Validation for rules completed successfully
[2025-10-17 15:05:09,933] [INFO] [spark_expectations] {expectations.p

[2025-10-17 15:05:14,952] [INFO] [spark_expectations] {writer.py:save_df_as_table:88} - finished writing records to table: se_user_target_temp


[2025-10-17 15:05:15,215] [INFO] [spark_expectations] {writer.py:save_df_as_table:99} - product_id is not set for table se_user_target_temp in tableproperties, setting it now


[2025-10-17 15:05:19,386] [INFO] [spark_expectations] {expectations.py:wrapper:401} - Read from temp table started


...

[2025-10-17 15:06:00,968] [INFO] [spark_expectations] {writer.py:save_df_as_table:88} - finished writing records to table: se_user_stats_detailed
[2025-10-17 15:06:00,969] [INFO] [spark_expectations] {writer.py:write_detailed_stats:498} - Writing metrics to the detailed stats table: se_user_stats_detailed, ended
[2025-10-17 15:06:00,969] [INFO] [spark_expectations] {writer.py:write_detailed_stats:498} - Writing metrics to the detailed stats table: se_user_stats_detailed, ended


[2025-10-17 15:06:01,345] [INFO] [spark_expectations] {writer.py:write_detailed_stats:505} - Writing metrics to the output custom table: {self._context.get_query_dq_output_custom_table_name}, started
[2025-10-17 15:06:01,346] [INFO] [spark_expectations] {writer.py:save_df_as_table:64} - _save_df_as_table started
[2025-10-17 15:06:01,349] [INFO] [spark_expectations] {writer.py:save_df_as_table:82} - Writing records to table: se_user_stats_querydq_output
[2025-10-17 15:06:01,346] [INFO] [spark_expectations] {writer.py:save_df_as_table:64} - _save_df_as_table started
[2025-10-17 15:06:01,349] [INFO] [spark_expectations] {writer.py:save_df_as_table:82} - Writing records to table: se_user_stats_querydq_output


[2025-10-17 15:06:07,028] [INFO] [spark_expectations] {writer.py:save_df_as_table:88} - finished writing records to table: se_user_stats_querydq_output
[2025-10-17 15:06:07,028] [INFO] [spark_expectations] {writer.py:write_detailed_stats:516} - Writing metrics to the output custom table: {self._context.get_query_dq_output_custom_table_name}, ended
[2025-10-17 15:06:07,029] [INFO] [spark_expectations] {writer.py:write_error_stats:769} - Streaming stats to kafka is disabled, hence skipping writing to kafka
[2025-10-17 15:06:07,028] [INFO] [spark_expectations] {writer.py:write_detailed_stats:516} - Writing metrics to the output custom table: {self._context.get_query_dq_output_custom_table_name}, ended
[2025-10-17 15:06:07,029] [INFO] [spark_expectations] {writer.py:write_error_stats:769} - Streaming stats to kafka is disabled, hence skipping writing to kafka


[2025-10-17 15:06:07,370] [INFO] [spark_expectations] {slack.py:send_notification:42} - Message posted successfully!


DataFrame[id: bigint, age: double, email: string, meta_dq_run_id: string, meta_dq_run_datetime: string]

In [41]:
# Let's test the Slack webhook directly to make sure it's working
import requests
import json
import urllib3

# Disable SSL warnings for testing (only for local development)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def test_slack_webhook(webhook_url):
    """Test if the Slack webhook is working"""
    test_message = {
        "text": "🧪 **Spark Expectations Test** - This is a test message from your notebook to verify Slack integration is working!"
    }
    
    try:
        # Try with SSL verification disabled for local testing
        response = requests.post(webhook_url, json=test_message, verify=False)
        if response.status_code == 200:
            print("✅ SUCCESS: Slack webhook test passed! You should see a message in your Slack channel.")
            return True
        else:
            print(f"❌ FAILED: Slack webhook test failed with status code: {response.status_code}")
            print(f"Response: {response.text}")
            return False
    except Exception as e:
        print(f"❌ ERROR: Failed to send test message to Slack: {str(e)}")
        return False

# Test the webhook
webhook_url = "https://hooks.slack.com/services/YOUR/WEBHOOK/URL"
print("Testing Slack webhook...")
webhook_works = test_slack_webhook(webhook_url)

if not webhook_works:
    print("\n⚠️  WARNING: The Slack webhook is not working.")
    print("Please check:")
    print("1. Your Slack webhook URL is correct")
    print("2. The webhook is still active in your Slack app settings")
    print("3. Your network allows HTTPS connections to hooks.slack.com")

Testing Slack webhook...
✅ SUCCESS: Slack webhook test passed! You should see a message in your Slack channel.
✅ SUCCESS: Slack webhook test passed! You should see a message in your Slack channel.


In [39]:
# Configure SSL settings for Spark Expectations in local environment
import ssl
import os

# For local development, configure SSL to be more permissive
# This is needed because Spark Expectations uses requests internally
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['REQUESTS_CA_BUNDLE'] = ''
os.environ['CURL_CA_BUNDLE'] = ''

# Also configure urllib3 to not verify SSL (for local development only)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("✅ Configured SSL settings for local development")
print("⚠️  Note: SSL verification is disabled for local testing only")

✅ Configured SSL settings for local development
⚠️  Note: SSL verification is disabled for local testing only


In [40]:
# Monkey patch requests to disable SSL verification for local development
import requests
from functools import wraps

# Store the original request methods
original_post = requests.post
original_get = requests.get
original_request = requests.request

def patch_request_method(original_method):
    @wraps(original_method)
    def patched_method(*args, **kwargs):
        # Disable SSL verification for local development
        kwargs['verify'] = False
        return original_method(*args, **kwargs)
    return patched_method

# Apply the patches
requests.post = patch_request_method(original_post)
requests.get = patch_request_method(original_get)
requests.request = patch_request_method(original_request)

print("✅ Patched requests library to disable SSL verification")
print("🔧 This allows Spark Expectations to send Slack notifications in local environment")

✅ Patched requests library to disable SSL verification
🔧 This allows Spark Expectations to send Slack notifications in local environment
